    Pré-processamento

In [6]:
import pandas as pd
import openpyxl
from sklearn.model_selection import train_test_split

In [7]:
#Pré Processamento

#Mapeamento de Area Metropolitana
def mapeamento(row):
    if row['EST_MSA'] == 'New York-Newark-Jersey City, NY-NJ-PA Metro Area':
        return 35620
    elif row['EST_MSA'] == 'Detroit-Warren-Dearborn, MI Metro Area':
        return 19820
    elif row['EST_MSA'] == 'Los Angeles-Long Beach-Anaheim, CA Metro Area':
        return 31080
    elif row['EST_MSA'] == 'Chicago-Naperville-Elgin, IL-IN-WI Metro Area':
        return 16980
    elif row['EST_MSA'] == 'Dallas-Fort Worth-Arlington, TX Metro Area':
        return 19100
    elif row['EST_MSA'] == 'Houston-The Woodlands-Sugar Land, TX Metro Area':
        return 26420
    elif row['EST_MSA'] == 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area':
        return 47900
    elif row['EST_MSA'] == 'Miami-Fort Lauderdale-Pompano Beach, FL Metro Area':
        return 33100
    elif row['EST_MSA'] == 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area':
        return 37980
    elif row['EST_MSA'] == 'Atlanta-Sandy Springs-Alpharetta, GA Metro Area':
        return 12060
    elif row['EST_MSA'] == 'Phoenix-Mesa-Chandler, AZ Metro Area':
        return 38060
    elif row['EST_MSA'] == 'Boston-Cambridge-Newton, MA-NH Metro Area':
        return 14460
    elif row['EST_MSA'] == 'San Francisco-Oakland-Berkeley, CA Metro Area':
        return 41860
    elif row['EST_MSA'] == 'Riverside-San Bernardino-Ontario, CA Metro Area':
        return 40140
    elif row['EST_MSA'] == 'Seattle-Tacoma-Bellevue, WA Metro Area':
        return 42660
    elif row['EST_MSA'] > 0:
        return row['EST_MSA']
    else:
        return 0

df = pd.read_excel('dataset.xlsx', skiprows=2)
df['EST_MSA'] = df.apply(mapeamento,axis=1)

#Remoção de mês de nascimento para padronização
df['TBIRTH_YEAR'] = df['TBIRTH_YEAR'].astype(str).str.split("-").str[0]

#Mapeamento de Região
def regionMapping(row):
    if row['EST_ST'] in (9,23,25,33,34,36,42,44,50):
        return 1
    elif row['EST_ST'] in (1,5,10,11,12,13,21,22,24,28,37,40,45,47,48,51,54):
        return 2
    elif row['EST_ST'] in (18,17,19,20,26,27,29,31,38,39,46,55):
        return 3
    elif row['EST_ST'] in (2,4,6,8,15,16,30,32,35,41,49,53,56):
        return 4
    else:
        return ""

df['REGION'] = df.apply(regionMapping,axis=1)

#Binarização remoto ou não
def remoteBinary(row):
    if row['TWDAYS_RESP'] in (1,3):
        return 1
    else:
        return 0

df['IS_RMT'] = df.apply(remoteBinary,axis=1)

df.to_excel('resultado.xlsx', index=False)
print("Feito")

Feito


Separação do database - Validação (30%) / Análise Exploratória (10%) / Teste (60%)

In [10]:
train, temp = train_test_split(df, test_size=0.4, stratify=df['IS_RMT'], random_state=42)
test, explore = train_test_split(temp, test_size=0.25, stratify=temp['IS_RMT'], random_state=42)

# Exportando excel de dados exploratórios p/ análise em Power BI
explore.to_excel('dados_exploratorios.xlsx', index=False)